# SKT-koBERT

https://github.com/SKTBrain/KoBERT

## Install & Library

In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import os

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})

# CPU or GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

## Model & Tokenizer

In [11]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [12]:
# 내장된 tokenizer와 sentencepiece 함수를 쓰는 듯
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        #self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i]) #+ (self.labels[i], ))

    def __len__(self):
        return (len(self.sentences))

In [14]:
## Setting parameters
max_len = 210 #210 #수정
batch_size = 16 # 64 #수정
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, #2 #수정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        classifier = self.classifier(pooler)
        
        if self.dr_rate:
            out = self.dropout(classifier)
  
        return pooler, out

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [17]:
def question_BERTClassifier(_dataloader):  
    for batch_id, (token_ids, valid_length, segment_ids) in tqdm_notebook(enumerate(_dataloader)):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length  

        embedding, probability = model.forward(token_ids, valid_length, segment_ids)
        embedding = np.array(embedding.tolist())
        probability = np.array(probability.tolist())

        if batch_id == 0:
            question_embedding = embedding
            question_probability = probability
        else:
            question_embedding = np.concatenate([question_embedding, embedding], axis=0)
            question_probability = np.concatenate([question_probability, probability], axis=0)

        del embedding, probability
        torch.cuda.empty_cache() # GPU 캐시 삭제
        
    # max index 추출
    question_class = np.argmax(question_probability, axis=1)
    
    return question_embedding, question_class

## Question Embedding & Classification

In [20]:
model_path = '/home/hyejeongeun/chatbot/Model/Model/'
base_path = '/home/hyejeongeun/chatbot/Data/QnA/'

In [21]:
# 모델 불러오기
model.load_state_dict(torch.load(model_path+'epoch_5_qna_sep_model_4.pt'))
# 모델 evaluation으로
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [22]:
# load data
df = pd.read_csv(base_path + 'qna_table_mecab.csv')

# label '기타' & NaN 제거
df = df[(~df['label'].isna())&(df['label']!='기타')]

# input list 생성
inputs = df['question_mecab'].astype(str).tolist()

# make dataset
data = BERTDataset(inputs, 0, tok, max_len, True, False)
_dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=5)

# embedding & classifier
question_embedding, question_class = question_BERTClassifier(_dataloader)

# dataframe 파일 생성
df_out = pd.DataFrame()
df_out['question_embedding'] = question_embedding.tolist()
df_out['question_class'] = question_class.tolist()

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

In [23]:
df.shape

(91672, 20)

In [24]:
df_out.shape

(91672, 2)

In [25]:
df_out['question_class'].value_counts()

1    56092
0    34013
3     1239
2      328
Name: question_class, dtype: int64

In [21]:
df['label'].value_counts()

상품    44904
배송    30148
교환     8357
환불     5780
반품     2483
Name: label, dtype: int64

In [37]:
df_

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,...,answer_clean,question_spellcheck,answer_spellcheck,question_mecab,answer_mecab,qna_hanspell_sep,qna_mecab_sep,num_label,question_embedding,question_class
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3076525,True,False,상품,배송 언제 되는걸까요...?,2021-03-18T15:31:16.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n유선상 연락 및 안내드렸습니다.\r\n...,...,안녕하세요 고객님 먼데이하우스입니다 유선상 연락 및 안내드렸습니다 감사합니다 1,배송 언제 되는 걸까요,안녕하세요 고객님 먼데이 하우스입니다 유선상 연락 및 안내드렸습니다 감사합니다 1,배송 언제 되,안녕 고객 먼데이 하우스 유선 연락 및 안 감사,배송 언제 되는 걸까요 [SEP] 안녕하세요 고객님 먼데이 하우스입니다 유선상 연락...,배송 언제 되 [SEP] 안녕 고객 먼데이 하우스 유선 연락 및 안 감사\t,0,"[0.6285900473594666, -0.016143934801220894, -0...",1
1,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 화이트,3076317,True,False,배송,구매 하였는데 배송 메모는 무시해주세요ㅠ 다른곳에서 산거 수정을 못했어요ㅠㅠ 빠른배...,2021-03-18T15:05:32.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n네 ~~주문확인되는데로 최대한 빠른 배...,...,안녕하세요 고객님 먼데이하우스입니다 네 주문확인되는데로 최대한 빠른 배송위해 최선을...,구매하였는데 배송 메모는 무시해주세요 다른 곳에서 산거 수정을 못했어요 빠른 배송 ...,안녕하세요 고객님 먼데이 하우스입니다 네 주문 확인되는 데로 최대한 빠른 배송 위해...,구매 배송 메모 무시 주 다른 곳 산거 수정 못 배송 부탁,안녕 고객 먼데이 하우스 네 주문 확인 데 최대한 배송 최선 다 하 감사,구매하였는데 배송 메모는 무시해주세요 다른 곳에서 산거 수정을 못했어요 빠른 배송 ...,구매 배송 메모 무시 주 다른 곳 산거 수정 못 배송 부탁 [SEP] 안녕 고객 먼...,1,"[0.20092280209064484, -0.32670801877975464, -0...",1
2,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 블랙,3073853,True,False,상품,조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요.. 육각 나사 렌...,2021-03-18T10:23:56.000+09:00,안녕하세요 고객님~~먼데이하우스입니다.\r\n따로 발송해드리도록 하겠습니다.\r\n...,...,안녕하세요 고객님 먼데이하우스입니다 따로 발송해드리도록 하겠습니다 감사합니다 1,조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요 육각 나사 렌치 ...,안녕하세요 고객님 먼데이 하우스입니다 따로 발송해드리도록 하겠습니다 감사합니다 1,조립 육각 나사 렌치 돌리 부분 다 조립 못 육각 나사 렌치 주 수 있,안녕 고객 먼데이 하우스 따로 발송 드리 하 감사,조립하다가 육각 나사 렌치 돌리는 부분이 다 갈려서 조립을 못해요 육각 나사 렌치 ...,조립 육각 나사 렌치 돌리 부분 다 조립 못 육각 나사 렌치 주 수 있 [SEP] ...,0,"[0.4521189332008362, 0.1962922364473343, -0.50...",1
3,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3070524,False,False,배송,주문해서 다음주 월요일(3/22)에 받고싶은데..언제 주문하면될까요?,2021-03-17T19:50:58.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n월요일 수령은 장담드리기 어렵습니다.주말...,...,안녕하세요 고객님 먼데이하우스입니다 월요일 수령은 장담드리기 어렵습니다 주말 휴무이...,주문해서 다음 주 월요일 3 22에 받고 싶은데 언제 주문하면 될까요,안녕하세요 고객님 먼데이 하우스입니다 월요일 수령은 장담 드리기 어렵습니다 주말 휴...,주문 다음 주 월요일 받 싶 언제 주문,안녕 고객 먼데이 하우스 월요일 수령 장담 드리 어렵 주말 휴무 이 때문 금요일 출...,주문해서 다음 주 월요일 3 22에 받고 싶은데 언제 주문하면 될까요 [SEP] 안...,주문 다음 주 월요일 받 싶 언제 주문 [SEP] 안녕 고객 먼데이 하우스 월요일 ...,1,"[0.39364686608314514, 0.23920387029647827, -0....",1
4,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3069196,True,False,상품,나사 너무 안들어가는데 원래 이렇게 나사가 안들어가나요? 힘을 줘도 절대 끝까지 안...,2021-03-17T16:57:32.000+09:00,안녕하세요~ 고객님 먼데이하우스입니다. \r\n저희 제품들이 원목 제품이라서 나사보...,...,안녕하세요 고객님 먼데이하우스입니다 저희 제품들이 원목 제품이라서 나사보다 구멍이 ...,나사 너무 안 들어가는데 원래 이렇게 나사가 안 들어가나요 힘을 줘도 절대 끝까지 ...,안녕하세요 고객님 먼데이 하우스입니다 저희 제품들이 원목 제품이라서 나사보다 구멍이...,나사 너무 안 들어가 원래 이렇게 나사 안 들어가 힘 절대 끝 안,안녕 고객 먼데이 하우스 저희 제품 원목 제품 이 나사 구멍 작 있 사용 지 하 현...,나사 너무 안 들어가는데 원래 이렇게 나사가 안 들어가나요 힘을 줘도 절대 끝까지 ...,나사 너무 안 들어가 원래 이렇게 나사 안 들어가 힘 절대 끝 안 [SEP] 안녕 ...,0,"[0.5652652978897095, 0.05006789788603783, -0.3...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91667,159367,바스 양변기 시공 (시공/철거/수거비 포함),NaN,1243972,False,False,배송,배송비는 따로 청구되나요?\n경기도 파주 야당동입니다,2020-05-23T09:20:33.000+09:00,"안녕하세요, 고객님. 노루페인트 하우홈인테리어 담당자입니다. \r\n우선 저희 시공...",...,안녕하세요 고객님 노루페인트 하우홈인테리어 담당자입니다 우선 저희 시공 상품에 관심...,배송비는 따로 청구되나요 경기도 파주 야당동입니다,안녕하세요 고객님 노루페인트 하우 홈 인테리어 담당자입니다 우선 저희 시공 상품에 ...,배송 비 따로 청구 경기도 파주 야당동,안녕 고객 노루 페인트 홈 인테리어 담당자 우선 저희 시공 상품 관심 주 감사 드리...,배송비는 따로 청구되나요 경기도 파주 야당동입니다 [SEP] 안녕하세요 고객님 노루...,배송 비 따로 청구 경기도 파주 야당동 [SEP] 안녕 고객 노루 페인트 홈 인테리...,1,"[0.6285900473594666, -0.0161439199000597, -0.7...",1
91668,159367,바스 양변기 시공 (시공/철거/수거비 포함),NaN,1147700,False,False,상품,변기교체하려는데요\n기존 변기 수거도 해주시고 기사님이 배송하시고 시공설치해주시는...,2020-05-04T15:42:10.000+09:00,"안녕하세요, 하우홈인테리어 담당자입니다. \r\n제품구매 확인 후 해피콜시 시공일정...",...,안녕하세요 하우홈인테리어 담당자입니다 제품구매 확인 후 해피콜시 시공일정에 대한 자...,변기 교체하려는데요 기존 변기 수거도 해주시고 기사님이 배송하시고 시공 설치해주시는...,안녕하세요 하우 홈 인테리어 담당자입니다 제품 구매 확인 후 해피콜 시 시공 일정에...,변기 교체 요 기존 변기 수거 

In [31]:
df_=pd.merge(df,df_out,left_index=True,right_index=True)

In [32]:
df_.shape

(91672, 22)

In [36]:
# csv 파일 내보내기
df_.to_csv(base_path+'question_embedding.csv', sep='\t', index=False)